# Qualitative Constraint Network Repairs





In [ ]:
import tarfile
import os
import urllib.request

url = "https://msioutis.gitlab.io/files/unsatdataset.tar.bz2"
filename = "unsatdataset.tar.bz2"
extract_folder = ".temp"

os.makedirs(extract_folder, exist_ok=True)

if not os.path.exists(os.path.join(extract_folder, "unsatdataset")):
    urllib.request.urlretrieve(url, filename)
    with tarfile.open(filename, "r:bz2") as tar:
        tar.extractall(path=extract_folder)
    os.remove(filename)

    print("Dataset downloaded and extracted.")
print("Files are in the folder: ", os.path.join(extract_folder, "unsatdataset"))

In [9]:
import re

def aspify_allen_relations(relations):
    rename_map = {'<': 'b', '>': 'bi', '=': 'eq'}
    return [rename_map.get(rel, rel) for rel in relations]

def parse_qcn_file(file_path):
    graphs = [] 
    current_graph = None 
    
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()

            if re.match(r'^\d+ #', line):
                if current_graph:
                    graphs.append(current_graph)
                
                instance_id = line.split()[0]  
                current_graph = {"instance": instance_id, "edges": []}
            elif re.match(r'^\d+\s+\d+\s+\(.*\)', line):
                parts = re.split(r'\s+|\(|\)', line)
                source = int(parts[0])
                target = int(parts[1])
                relations = parts[3:-1]
                relations = aspify_allen_relations([x.lower() for x in relations if x])

                current_graph["edges"].append({
                    "from": source,
                    "to": target,
                    "relations": relations
                })
    
    if current_graph:
        graphs.append(current_graph)
    return graphs

def qcn2asp(qcn):
    asp = []
    for edge in qcn["edges"]:
        rel = "(" + ";".join(edge['relations']) + ")"
        asp.append(f"constraint({edge['from']}, {edge['to']}, {rel}).")
    return asp



In [ ]:
SATISFY = "encoding/satisfy.lp"
COMPLETE = "encoding/graph/complete.lp"
REPAIR = "encoding/repair.lp"
CALCULI = "encoding/calculi/rcc8.lp"

# Example usage:
file_path = ".temp/unsatdataset/rcc8/100xincon_n30_d8_rcc8.csp"  # Replace with the actual filename
qcn_data = parse_qcn_file(file_path)

for qcn_instance in qcn_data[0:10]:
    instance_file_path = os.path.join(extract_folder, "instance.lp")
    with open(instance_file_path, "w") as f:
        f.write("\n".join(qcn2asp(qcn_instance)))

    !time timeout 60 clingo 0 --parallel-mode 2,compete {instance_file_path} {COMPLETE} {REPAIR} {CALCULI} {SATISFY}